Курсовой проект для курса "Python для Data Science"

Материалы к проекту (файлы):
train.csv
test.csv

Задание:
Используя данные из train.csv, построить
модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели предсказать
цены для квартир из файла test.csv.

Целевая переменная:
Price

Метрика:
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

Сдача проекта:
1. Прислать в раздел Задания Урока 10 ("Вебинар. Консультация по итоговому проекту")
ссылку на программу в github (программа должна содержаться в файле Jupyter Notebook 
с расширением ipynb). (Pull request не нужен, только ссылка ведущая на сам скрипт).
2. Приложить файл с названием по образцу SShirkin_predictions.csv
с предсказанными ценами для квартир из test.csv (файл должен содержать два поля: Id, Price).
В файле с предсказаниями должна быть 5001 строка (шапка + 5000 предсказаний).

Сроки и условия сдачи:
Дедлайн: сдать проект нужно в течение 72 часов после начала Урока 10 ("Вебинар. Консультация по итоговому проекту").
Для успешной сдачи должны быть все предсказания (для 5000 квартир) и R2 должен быть больше 0.6.
При сдаче до дедлайна результат проекта может попасть в топ лучших результатов.
Повторная сдача и проверка результатов возможны только при условии предыдущей неуспешной сдачи.
Успешный проект нельзя пересдать в целях повышения результата.
Проекты, сданные после дедлайна или сданные повторно, не попадают в топ лучших результатов, но можно узнать результат.
В качестве итогового результата берется первый успешный результат, последующие успешные результаты не учитываются.

Примечание:
Все файлы csv должны содержать названия полей (header - то есть "шапку"),
разделитель - запятая. В файлах не должны содержаться индексы из датафрейма.

Рекомендации для файла с кодом (ipynb):
1. Файл должен содержать заголовки и комментарии
2. Повторяющиеся операции лучше оформлять в виде функций
3. Не делать вывод большого количества строк таблиц (5-10 достаточно)
4. По возможности добавлять графики, описывающие данные (около 3-5)
5. Добавлять только лучшую модель, то есть не включать в код все варианты решения проекта
6. Скрипт проекта должен отрабатывать от начала и до конца (от загрузки данных до выгрузки предсказаний)
7. Весь проект должен быть в одном скрипте (файл ipynb).
8. При использовании статистик (среднее, медиана и т.д.) в качестве признаков,
лучше считать их на трейне, и потом на валидационных и тестовых данных не считать 
статистики заново, а брать их с трейна. Если хватает знаний, можно использовать кросс-валидацию,
но для сдачи этого проекта достаточно разбить данные из train.csv на train и valid.
9. Проект должен полностью отрабатывать за разумное время (не больше 10 минут),
поэтому в финальный вариант лучше не включать GridSearch с перебором 
большого количества сочетаний параметров.
10. Допускается применение библиотек Python и моделей машинного обучения,
которые были в курсе Python для Data Science. Градиентный бустинг изучается
в последующих курсах, поэтому в этом проекте его применять не следует.
Самая сложная из допустимых моделей - RandomForestRegressor из sklearn.

In [1]:
#Подключаем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.metrics import r2_score as r2, accuracy_score
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import KFold, GridSearchCV
#from xgboost import XGBRegressor, XGBClassifier
#from lightgbm import LGBMRegressor, LGBMClassifier
import seaborn as sns

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
plt.rcParams.update({'font.size': 14})

In [5]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print('Train R2:\t' + str(round(r2(train_true_values, train_pred_values), 3)))
    print('Test R2:\t' + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize = (18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')
    plt.show()

In [6]:
def show_dstr(train_df, column):
    plt.figure(figsize = (16,8))
    
    train_df[column].hist(bins=18)
    plt.ylable('count')
    plt.xlable(column)
    
    plt.title('Target distribution')
    plt.show()
    return None
    

# Пути к директориям файла

In [7]:
TRAIN_DATASET_PATH ='train.csv'
TEST_DATASET_PATH ='test.csv'

In [8]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
print(train_df.shape)
train_df.head()

(10000, 20)


,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [9]:
train_df.isna().sum()

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4798
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
dtype: int64

In [10]:
train_df.nunique()

Id               10000
DistrictId         205
Rooms                9
Square           10000
LifeSquare        7887
KitchenSquare       58
Floor               33
HouseFloor          44
HouseYear           97
Ecology_1          129
Ecology_2            2
Ecology_3            2
Social_1            51
Social_2           142
Social_3            30
Healthcare_1        79
Helthcare_2          7
Shops_1             16
Shops_2              2
Price            10000
dtype: int64

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

Базовая модель. Выбрасываем признаки.

In [14]:
train_df.replace({'Ecology_2':{'A':0, 'B':1}, 'Ecology_3':{'A':0, 'B':1},'Shops_2':{'A':0, 'B':1}}, inplace=True)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(train_df.set_index('Id').drop(['LifeSquare', 'Healthcare_1', 'Price'], axis= 'columns'),
                                                   train_df['Price'], test_size =0.2, random_state=22)

In [31]:
#Модель основанная на признаках, где нет пропущеных значений
model_RFR = RandomForestRegressor(max_depth=12, n_estimators=200, random_state=42)
model_RFR.fit(X_train, y_train)
first_predict_train = model_RFR.predict(X_train)
first_predict = model_RFR.predict(X_test)
r2(y_test, first_predict)
print('r2_train = {r2}'.format(r2 = r2(y_train,first_predict_train)))
print('r2_test = {r2}'.format(r2 = r2(y_test,first_predict)))

r2_train = 0.8990576295478698
r2_test = 0.7169781652447873


In [32]:
X_test

,DistrictId,Rooms,Square,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
Id,,,,,,,,,,,,,,,,
10376,23,1.0,39.392675,1.0,9,17.0,1977,0.075779,1,1,6,1437,3,0,2,1
1627,45,3.0,74.970211,1.0,9,9.0,2014,0.195781,1,1,23,5212,6,3,2,1
8296,146,1.0,86.386754,0.0,37,38.0,2012,0.236108,1,1,16,3893,27,3,10,1
2666,103,3.0,56.072701,5.0,6,5.0,1960,0.151346,1,0,32,5889,10,4,1,1
10824,200,1.0,42.167687,8.0,11,17.0,1997,0.000000,1,1,33,7425,1,2,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6928,11,2.0,49.176900,6.0,9,9.0,1973,0.038693,1,1,28,6533,1,2,5,1
7059,52,1.0,41.404988,1.0,15,17.0,1977,0.371149,1,1,34,7065,1,2,5,1
4638,47,3.0,89.822947,10.0,13,24.0,2010,0.101872,1,1,23,4583,3,3,3,1


In [33]:
test = pd.read_csv(TEST_DATASET_PATH)
test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


TypeError: r2_score() missing 1 required positional argument: 'y_pred'

In [38]:
# Первое предсказание базовой модели на тестовых файлах
test.replace({'Ecology_2':{'A':0, 'B':1}, 'Ecology_3':{'A':0, 'B':1},'Shops_2':{'A':0, 'B':1}}, inplace=True)
test_first_predict = model_RFR.predict(test.set_index('Id').drop(['LifeSquare', 'Heаlthcare_1'], axis= 'columns'))

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [39]:
test['Price'] = test_first_predict
test.head()

NameError: name 'test_first_predict' is not defined

In [ ]:
test[['Id','Price']].to_csv('first_predict_2021-04-15_01.csv', index=False)

In [ ]:
test.set_index('Id').drop(['LifeSquare', 'Healthcare_1', 'Price'], axis= 'columns').columns.tolist()

In [ ]:
test[['Id','Price']].to_csv('first_predict_2408.csv', index=False)

In [ ]:
f_importance = pd.DataFrame()
f_importance['name'] = test.set_index('Id').drop['LifeSquare', 'Healthcare_1', 'Price']axis= 'columns').columns.tolist()
f_importance['values'] = model_RFR.feature_importances_
f_importance.sort_values('values', ascending = False).reset_index(drop=True)

In [41]:
#Заменим DistrictId на медианную цену дома в конкретном районе
median_price_by_id = train_df.groupby('DistrictId')[['Price']].median().reset_index()
median_price_by_id

,DistrictId,Price
0,0,165963.054142
1,1,183663.443595
2,2,208539.501373
3,3,169094.013281
4,4,278639.482329
...,...,...
200,202,394150.861857
201,205,220501.566180
202,207,426186.409334
203,208,431137.654083


In [40]:
train_df = pd.merge(train_df, median_price_by_id, how='left', on='DistrictId'
                   train_df)

SyntaxError: invalid syntax (<ipython-input-40-4b5566154bc5>, line 2)